# 다운로드

In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.init
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 6 files to the new cache system


  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\USER\.conda\envs\dacon_2022\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

# 데이터 함 훑어봅시다

In [6]:
data_dir = 'C:\\Dacon_2022\\data\\'
df_train = pd.read_csv(data_dir+'train.csv')
df_test = pd.read_csv(data_dir+'test.csv')
df_submit = pd.read_csv(data_dir+'sample_submission.csv')
display(df_train.head())
display(df_test.head())
display(df_submit.head())

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식


,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...


,id,cat3
0,TEST_00000,한식
1,TEST_00001,한식
2,TEST_00002,한식
3,TEST_00003,한식
4,TEST_00004,한식


## cat#이 어떻게 되어있나 봅시다

In [7]:
df_train['cat1'].value_counts()

인문(문화/예술/역사)    5614
음식              4912
레포츠             2611
자연              1719
숙박              1434
쇼핑               696
Name: cat1, dtype: int64

In [8]:
df_train['cat3'].unique()

array(['항구/포구', '골프', '한식', '야영장,오토캠핑장', '섬', '일반축제', '유적지/사적지', '일식',
       '자연휴양림', '전시관', '관광단지', '모텔', '스키(보드) 렌탈샵', '바/까페', '채식전문점',
       '한옥스테이', '컨벤션', '성', '중식', '안보관광', '전문상가', '이색거리', '해수욕장', '5일장',
       '미술관/화랑', '공원', '상설시장', '박물관', '도서관', '공연장', '농.산.어촌 체험', '복합 레포츠',
       '펜션', '산', '기념탑/기념비/전망대', '기념관', '자동차경주', '자전거하이킹', '희귀동.식물',
       '유명건물', '수목원', '유원지', '사찰', '강', '자연생태관광지', '박람회', '문화전수시설', '계곡',
       '폭포', '서양식', '종교성지', '동상', '기타행사', '고택', '국립공원', '게스트하우스', '썰매장',
       '요트', '문화원', '래프팅', '승마', '공예,공방', '트래킹', '온천/욕장/스파', '수련시설',
       '해안절경', '패밀리레스토랑', '기암괴석', '기타', '홈스테이', '민물낚시', '콘도미니엄', '민박',
       '테마공원', '유람선/잠수함관광', '바다낚시', '수영', '백화점', '호수', '카약/카누', '외국문화원',
       '면세점', '다리/대교', '카지노', '분수', '동굴', '도립공원', '윈드서핑/제트스키', '특산물판매점',
       '유스호스텔', '고궁', '서비스드레지던스', '민속마을', '전통공연', '문', '식음료', '번지점프',
       '터널', '문화관광축제', '카트', '클럽', '발전소', '대중콘서트', 'MTB', '군립공원', '스케이트',
       '대형서점', '생가', '스카이다이빙', '영화관', '헹글라이딩/패러글라이딩', '스노쿨링/스킨스쿠버다이

### cat1: 인문(문화/예술/역사)

In [9]:
df_inmun = df_train[df_train['cat1']=='인문(문화/예술/역사)']
df_inmun['cat2'].value_counts()

역사관광지     1571
문화시설      1220
휴양관광지      889
체험관광지      805
축제         569
건축/조형물     328
공연/행사      139
산업관광지       93
Name: cat2, dtype: int64

In [10]:
pd.pivot_table(df_inmun, index=['cat1', 'cat2', 'cat3'], aggfunc='count')

id  img_path  overview
cat1         cat2   cat3                                
인문(문화/예술/역사) 건축/조형물 기념탑/기념비/전망대  148       148       148
                    다리/대교         41        41        41
                    동상            18        18        18
                    분수             8         8         8
                    유명건물         105       105       105
                    터널             8         8         8
             공연/행사  기타행사          71        71        71
                    대중콘서트          2         2         2
                    뮤지컬            2         2         2
                    박람회           29        29        29
                    연극             5         5         5
                    전통공연           6         6         6
                    컨벤션           22        22        22
                    클래식음악회         2         2         2
             문화시설   공연장          120       120       120
                    기념관           64        64        64
                    대형서점          17        17        17
                    도서관           92        92        92
                    문화원          113       113       113
                    문화전수시설        26        26        26
                    미술관/화랑       204       204       204
                    박물관          310       310       310
                    영화관            4         4         4
                    외국문화원          7         7         7
                    전시관          243       243       243
                    컨벤션센터         11        11        11
                    학교             9         9         9
             산업관광지  기타            79        79        79
                    발전소            4         4         4
                    식음료           10        10        10
             역사관광지  고궁            36        36        36
                    고택            92        92        92
                    문             23        23        23
                    민속마을          76        76        76
                    사찰           495       495       495
                    생가            43        43        43
                    성             85        85        85
                    안보관광          34        34        34
                    유적지/사적지      622       622       622
                    종교성지          65        65        65
             체험관광지  농.산.어촌 체험    332       332       332
                    이색거리         467       467       467
                    이색체험           6         6         6
             축제     문화관광축제        20        20        20
                    일반축제         549       549       549
             휴양관광지  공원           271       271       271
                    관광단지         126       126       126
                    온천/욕장/스파      53        53        53
                    유람선/잠수함관광     30        30        30
                    유원지          211       211       211
                    이색찜질방         11        11        11
                    테마공원         172       172       172
                    헬스투어          15        15        15

### cat1: 음식

In [11]:
df_food = df_train[df_train['cat1']=='음식']
df_food['cat2'].value_counts()

음식점    4912
Name: cat2, dtype: int64

In [12]:
pd.pivot_table(df_food, index=['cat1', 'cat2', 'cat3'], aggfunc='count')

id  img_path  overview
cat1 cat2 cat3                             
음식   음식점  바/까페      785       785       785
          서양식       226       226       226
          일식        151       151       151
          중식        209       209       209
          채식전문점      36        36        36
          클럽          6         6         6
          패밀리레스토랑    61        61        61
          한식       3438      3438      3438

### cat1: 레포츠

In [13]:
df_reports = df_train[df_train['cat1']=='레포츠']
df_reports['cat2'].value_counts()

육상 레포츠    2288
수상 레포츠     277
복합 레포츠      24
항공 레포츠      11
레포츠소개       11
Name: cat2, dtype: int64

In [14]:
pd.pivot_table(df_reports, index=['cat1', 'cat2', 'cat3'], aggfunc='count')

id  img_path  overview
cat1 cat2   cat3                                    
레포츠  레포츠소개  수상레포츠             11        11        11
     복합 레포츠 복합 레포츠            24        24        24
     수상 레포츠 래프팅               18        18        18
            민물낚시             119       119       119
            바다낚시              57        57        57
            수영                27        27        27
            스노쿨링/스킨스쿠버다이빙      7         7         7
            요트                12        12        12
            윈드서핑/제트스키         27        27        27
            카약/카누             10        10        10
     육상 레포츠 ATV                4         4         4
            MTB                2         2         2
            골프               179       179       179
            번지점프               4         4         4
            빙벽등반               4         4         4
            사격장                9         9         9
            수련시설              95        95        95
            스케이트              12        12        12
            스키(보드) 렌탈샵        60        60        60
            스키/스노보드            6         6         6
            승마                21        21        21
            썰매장               27        27        27
            야영장,오토캠핑장       1558      1558      1558
            인라인(실내 인라인 포함)     2         2         2
            자동차경주             35        35        35
            자전거하이킹            13        13        13
            카지노                4         4         4
            카트                 7         7         7
            트래킹              246       246       246
     항공 레포츠 스카이다이빙             2         2         2
            헹글라이딩/패러글라이딩       9         9         9

### cat1: 자연

In [15]:
df_nature = df_train[df_train['cat1']=='자연']
df_nature['cat2'].value_counts()

자연관광지    1578
관광자원      141
Name: cat2, dtype: int64

In [16]:
pd.pivot_table(df_nature, index=['cat1', 'cat2', 'cat3'], aggfunc='count')

id  img_path  overview
cat1 cat2  cat3                            
자연   관광자원  기암괴석      53        53        53
           희귀동.식물    88        88        88
     자연관광지 강        104       104       104
           계곡       160       160       160
           국립공원      21        21        21
           군립공원      11        11        11
           도립공원      15        15        15
           동굴        20        20        20
           등대        24        24        24
           산        239       239       239
           섬        111       111       111
           수목원       95        95        95
           약수터       17        17        17
           자연생태관광지  207       207       207
           자연휴양림    118       118       118
           폭포        40        40        40
           항구/포구    134       134       134
           해수욕장     207       207       207
           해안절경      25        25        25
           호수        30        30        30

### cat1: 숙박

In [17]:
df_sookbak = df_train[df_train['cat1']=='숙박']
df_sookbak['cat2'].value_counts()

숙박시설    1434
Name: cat2, dtype: int64

In [18]:
pd.pivot_table(df_sookbak, index=['cat1', 'cat2', 'cat3'], aggfunc='count')

id  img_path  overview
cat1 cat2 cat3                             
숙박   숙박시설 게스트하우스     83        83        83
          모텔        340       340       340
          민박         39        39        39
          서비스드레지던스   12        12        12
          유스호스텔      20        20        20
          콘도미니엄      34        34        34
          펜션        445       445       445
          한옥스테이     404       404       404
          홈스테이       57        57        57

### cat1: 쇼핑

In [19]:
df_shopping = df_train[df_train['cat1']=='쇼핑']
df_shopping['cat2'].value_counts()

쇼핑    696
Name: cat2, dtype: int64

In [20]:
pd.pivot_table(df_shopping, index=['cat1', 'cat2', 'cat3'], aggfunc='count')

id  img_path  overview
cat1 cat2 cat3                           
쇼핑   쇼핑   5일장     165       165       165
          공예,공방    41        41        41
          면세점       6         6         6
          백화점       4         4         4
          상설시장    270       270       270
          전문상가    173       173       173
          특산물판매점   37        37        37

## 다시 데이터프레임

In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16986 entries, 0 to 16985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16986 non-null  object
 1   img_path  16986 non-null  object
 2   overview  16986 non-null  object
 3   cat1      16986 non-null  object
 4   cat2      16986 non-null  object
 5   cat3      16986 non-null  object
dtypes: object(6)
memory usage: 796.3+ KB


### 샘플링해서 전처리

In [23]:
sample = df_train.loc[42:4242]
sample

,id,img_path,overview,cat1,cat2,cat3
42,TRAIN_00042,./image/train/TRAIN_00042.jpg,경기도 용인자연휴양림 야영장은 울창한 숲속에 각종 편의시설이 잘 갖춰진 캠핑장이다....,레포츠,육상 레포츠,"야영장,오토캠핑장"
43,TRAIN_00043,./image/train/TRAIN_00043.jpg,20년 전통의 샤브샤브 전문점이다. 대표메뉴는 샤브샤브다. 서울특별시 영등포구에 있...,음식,음식점,한식
44,TRAIN_00044,./image/train/TRAIN_00044.jpg,지하철 3호선 경복궁역에 위치한 서울메트로미술관은 화강석의 수려함과 아치형의 웅장함...,인문(문화/예술/역사),문화시설,미술관/화랑
45,TRAIN_00045,./image/train/TRAIN_00045.jpg,김포 황토테마파크는 경기도 김포시 하성면에 자리 잡고 있다. 김포시청을 기점으로 전...,레포츠,육상 레포츠,"야영장,오토캠핑장"
46,TRAIN_00046,./image/train/TRAIN_00046.jpg,운문산 군립공원 소머리 야영장은 경북 청도군 운문면에 자리 잡고 있다. 청도군청을 ...,레포츠,육상 레포츠,"야영장,오토캠핑장"
...,...,...,...,...,...,...
4238,TRAIN_04238,./image/train/TRAIN_04238.jpg,소강고택은 경북 봉화 바래미마을에 자리한 100여 년 된 한옥이다. 의성 김 씨 집...,숙박,숙박시설,한옥스테이
4239,TRAIN_04239,./image/train/TRAIN_04239.jpg,부천문화원은 1966년에 설립되어 지금까지 40여 년 동안 부천의 전통 민속의 보존...,인문(문화/예술/역사),문화시설,문화원
4240,TRAIN_04240,./image/train/TRAIN_04240.jpg,강원도 홍천에서 구매한 육즙이 풍성하고 마블링이 우수한 한우 꽃등심만을 숙성시켜 참...,음식,음식점,한식
4241,TRAIN_04241,./image/train/TRAIN_04241.jpg,오죽헌과 강릉시립박물관으로 구분된다. 오죽헌은 조선초기의 건축물로 건축사적인 면에서...,인문(문화/예술/역사),문화시설,박물관


In [ ]:
txt_smp = sample['overview'].tolist()
txt_smp

['경기도 용인자연휴양림 야영장은 울창한 숲속에 각종 편의시설이 잘 갖춰진 캠핑장이다. 용인시에서 운영하는 자연휴양림 속에 위치하며 지친 몸과 마음에 휴식을 제공하기 충분하다. 울창한 소나무 숲과 낙엽송 숲, 밤나무 숲 등 자연경관을 그대로 보존한 휴양림 속에 데크를 비롯해 화장실, 샤워실, 개수대 등 편의시설이 잘 갖춰져 있다. 용인자연휴양림 내에 에코어드벤처, 짚라인 등 다양한 액티비티 프로그램을 이용할 수 있다.',
 '20년 전통의 샤브샤브 전문점이다. 대표메뉴는 샤브샤브다. 서울특별시 영등포구에 있는 한식전문점이다.',
 '지하철 3호선 경복궁역에 위치한 서울메트로미술관은 화강석의 수려함과 아치형의 웅장함을 겸비하였고 시민 누구나 접근이 편리하고 무료로 이용할 수 있는 도심 속의 격조 높은 문화공간이다. 1986년 지하철 3호선 개통과 함께 지난 20년 간 역사적 문화시설인 경복궁을 중심으로 한 문화예술지역의 초입에서 대중적인 문화공간으로 자리하고 있는 미술관의 시설을 개선하여 더욱 편리하게 관람 할 수 있는 예술 공간이 되도록 하였다. 특히, 2005년, 2006년 및 2008년 등 3차례에 걸친 리모델링을 통해 안내데스크, 전시면 조명시설, CCTV 및 DVR, LED 전광판, 강화 GLASS DOOR, 냉난방시설등을 설치하여 그 품격을 더욱 높였을 뿐만 아니라, 최첨단 무인경비시스템 설치로 작품에 대한 훼손 및 도난방지에도 심혈을 기울였다. 또한, 2008년부터 전국규모의 서울메트로미술공모전을 개최함으로써 미술애호가들에게 참여의 기회를 제공함은 물론 최고 수준의 공공미술관으로써 거듭나고 있다.<br>',
 '김포 황토테마파크는 경기도 김포시 하성면에 자리 잡고 있다. 김포시청을 기점으로 전류리포구 쪽으로 방향을 잡고 바리미사거리와 봉성리 삼거리를 거치면 닿는다. 도착까지 걸리는 시간은 20분 안팎이다. 10,000㎡에 이르는 이곳은 김포에서 유일하게 작은 계곡이 흐르는 캠핑장이다. 주변은 산으로 둘러싸여 자연 친화적이기도 하다. 캠핑장에는 오토캠

In [ ]:
txt_smp[2].split('<br>')

['지하철 3호선 경복궁역에 위치한 서울메트로미술관은 화강석의 수려함과 아치형의 웅장함을 겸비하였고 시민 누구나 접근이 편리하고 무료로 이용할 수 있는 도심 속의 격조 높은 문화공간이다. 1986년 지하철 3호선 개통과 함께 지난 20년 간 역사적 문화시설인 경복궁을 중심으로 한 문화예술지역의 초입에서 대중적인 문화공간으로 자리하고 있는 미술관의 시설을 개선하여 더욱 편리하게 관람 할 수 있는 예술 공간이 되도록 하였다. 특히, 2005년, 2006년 및 2008년 등 3차례에 걸친 리모델링을 통해 안내데스크, 전시면 조명시설, CCTV 및 DVR, LED 전광판, 강화 GLASS DOOR, 냉난방시설등을 설치하여 그 품격을 더욱 높였을 뿐만 아니라, 최첨단 무인경비시스템 설치로 작품에 대한 훼손 및 도난방지에도 심혈을 기울였다. 또한, 2008년부터 전국규모의 서울메트로미술공모전을 개최함으로써 미술애호가들에게 참여의 기회를 제공함은 물론 최고 수준의 공공미술관으로써 거듭나고 있다.',
 '']

In [ ]:
import re
class Preprocessor:
  def __init__(self, sample):
    self.sample = sample
    self.text_init = sample['overview'].tolist()
    self.text = self.text_init.copy()
    # print(self.text_init)
  

  def del_special_char(self):
    for i, str in enumerate(self.text):
      loop = True

      while(loop):
        loop = False
        if '<' in str:
          try:
            sp_char = str[str.find('<'):str.find('>')+1]
            if re.compile('[ㄱ-ㅎ ㅏ-ㅣ 가-힣]+').sub('', str)!=str:
              print(i, sp_char)
              str = ' '.join(str.split(sp_char))
              self.text[i] = str
              loop = True
          except: pass
        if '(' in str:
          try:
            sp_char = str[str.find('('):str.find(')')+1]
            if re.compile('[ㄱ-ㅎ ㅏ-ㅣ 가-힣]+').sub('', str)!=str:
              print(i, sp_char)
              str = ' '.join(str.split(sp_char))
              self.text[i] = str
              loop = True
          except: pass
        if '[' in str:
          try:
            sp_char = str[str.find('['):str.find(']')+1]
            if re.compile('[ㄱ-ㅎ ㅏ-ㅣ 가-힣]+').sub('', str)!=str:
              print(i, sp_char)
              str = ' '.join(str.split(sp_char))
              self.text[i] = str
              loop = True
          except: pass
        if '\n' in str:
          print('Deleted line-changer')
          str = ' '.join(str.split('\n'))
          self.text[i] = str
          loop = True
  

  def cut_by_sentence(self):
    def pass_no_char(lst):
      new_lst = []
      for chr in lst:
        if chr == '': pass
        else: new_lst.append(chr)
      return new_lst

    def make_up_space(txt_list):
      new_list = []
      for txt in txt_list:
        txt = ' '.join(txt.split(' '))
        # print(txt)
        new_list.append(txt)
      return new_list

    self.text_listed = []
    for i, str in enumerate(self.text):
      self.text_listed.append(make_up_space(pass_no_char(str.split('.'))))
    # print(self.text_listed)

In [ ]:
smp_process = Preprocessor(sample)
smp_process.del_special_char()
smp_process.cut_by_sentence()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1377 (1842)
1377 (1845)
1378 <br>
1378 (태종 7)
1378 (賢儒)
1378 (光風樓)
1378 (東齋)
1378 (西齋)
1378 (內三門)
1378 (五聖)
1378 (宋朝四賢)
1378 (十八賢)
1378 (釋奠)
1378 (奉行)
1378 (典校)
1378 (掌議)
1378 (庫舍)
Deleted line-changer
Deleted line-changer
1382 <br>
1383 <br>
1387 <br />
1387 (花寺)
1387 [花寺]
1387 <br>
1387 (꽃절)
1387 (石造臺座)
1387 (擧身光)
1387 (石井)
1387 (龜石)
1387 (서울대규정각)
1387 
1388 <br>
1388 (더비에스호텔)
Deleted line-changer
1389 <br>
1392 <br>
1394 <br />
1394 (꽃동산)
1394 <br>
1395 <br>
1396 <br />
1396 <br>
1397 <br>
Deleted line-changer
Deleted line-changer
1399 <br />
1399 (중종 25년)
1399 <br>
1399 (玄仁福)
1399 (南賃麟)
1399 (康基德)
1399 (開水)
1399 (영조 5년)
1399 (比安縣 輿地勝覽)
1399 (古開天池)
1399 (開天池)
1400 <br>
1400 (까치구멍)
1400 (코클)
1400 (화터)
1400 (항아리처럼 싸리로 엮어 만든 저장용구)
1400 (김치등을 담아두던 나무로 만든 통)
1400 (눈이 쌓였을 때 짚신위에 덧신어 다니기 편리하게 만든 살피)
1400 (새끼 따위로 촘촘히 엮어 멜끈을 달아 물건을 나르는데 씀)
1401 <br />
1401 (長書)
1401 <br>
1401 (제1차 유림단 사건)
1401 (1973.10.03. 건립)
1403 <br>
1403 (文正公)
1403 (숙종 16)
1403 (정조 23)

In [ ]:
smp_process.text_listed[0]

['경기도 용인자연휴양림 야영장은 울창한 숲속에 각종 편의시설이 잘 갖춰진 캠핑장이다',
 ' 용인시에서 운영하는 자연휴양림 속에 위치하며 지친 몸과 마음에 휴식을 제공하기 충분하다',
 ' 울창한 소나무 숲과 낙엽송 숲, 밤나무 숲 등 자연경관을 그대로 보존한 휴양림 속에 데크를 비롯해 화장실, 샤워실, 개수대 등 편의시설이 잘 갖춰져 있다',
 ' 용인자연휴양림 내에 에코어드벤처, 짚라인 등 다양한 액티비티 프로그램을 이용할 수 있다']

In [ ]:
pd.DataFrame(smp_process.text_listed)

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,경기도 용인자연휴양림 야영장은 울창한 숲속에 각종 편의시설이 잘 갖춰진 캠핑장이다,용인시에서 운영하는 자연휴양림 속에 위치하며 지친 몸과 마음에 휴식을 제공하기 충분하다,"울창한 소나무 숲과 낙엽송 숲, 밤나무 숲 등 자연경관을 그대로 보존한 휴양림 속...","용인자연휴양림 내에 에코어드벤처, 짚라인 등 다양한 액티비티 프로그램을 이용할 수 있다",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,20년 전통의 샤브샤브 전문점이다,대표메뉴는 샤브샤브다,서울특별시 영등포구에 있는 한식전문점이다,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,지하철 3호선 경복궁역에 위치한 서울메트로미술관은 화강석의 수려함과 아치형의 웅장함...,1986년 지하철 3호선 개통과 함께 지난 20년 간 역사적 문화시설인 경복궁을 ...,"특히, 2005년, 2006년 및 2008년 등 3차례에 걸친 리모델링을 통해 안...","또한, 2008년부터 전국규모의 서울메트로미술공모전을 개최함으로써 미술애호가들에게...",,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,김포 황토테마파크는 경기도 김포시 하성면에 자리 잡고 있다,김포시청을 기점으로 전류리포구 쪽으로 방향을 잡고 바리미사거리와 봉성리 삼거리를 ...,도착까지 걸리는 시간은 20분 안팎이다,"10,000㎡에 이르는 이곳은 김포에서 유일하게 작은 계곡이 흐르는 캠핑장이다",주변은 산으로 둘러싸여 자연 친화적이기도 하다,"캠핑장에는 오토캠핑 30면, 임대텐트 3면이 있다",모든 사이트에서 전기와 화로 사용이 가능하다,"단, 전기 사용 시 600W 미만의 제품만 이용할 수 있다","여름에는 수영장, 겨울에는 얼음 썰매장을 운영하며, 황토 체험장, 자연학습장 등도...","매점에서는 일반 편의점 용품을 판매하고, 관리소에서는 텐트 등 캠핑용품을 대여해 준다",...,None,None,None,None,None,None,None,None,None,None
4,운문산 군립공원 소머리 야영장은 경북 청도군 운문면에 자리 잡고 있다,"청도군청을 기점으로 약 40km 거리에 있으며, 자동차를 타고 청매로, 청려로, ...",도착까지 소요 시간은 60분가량이다,이곳은 도로와 인접해 접근이 쉽고 주변이 산에 둘러싸여 있다,캠핑장 앞으로는 계곡물이 넓게 흐르고 있고 입구에 들어서면 이곳의 상징인 거대한 ...,캠핑장에는 맨흙으로 이뤄진 일반 야영장 112면을 마련했다,"사이트 크기와 개수는 가로 6m 세로 7m 57면, 가로 8m 세로 10m 55면...",전기사용이 가능하고 개인 트레일러와 카라반 입장은 불가능하다,"캠핑장은 사계절 내내 운영하며, 예약은 현장 접수로만 받는다","캠핑장 인근에는 국립 운문산 자연휴양림, 운문사 등 여행지가 많아 연계 관광이 손쉽다",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,소강고택은 경북 봉화 바래미마을에 자리한 100여 년 된 한옥이다,"의성 김 씨 집성촌인 바래미마을에는 고택이 많이 남아 있는데, 소강고택은 역사와 ...",경상도 지방의 명망 높은 부호이자 애국지사였던 남호 김뢰식 선생이 그의 둘째 아들...,김뢰식 선생은 상해임시정부가 군자금을 모을 때 전 재산을 담보로 돈을 빌려 힘을 ...,소강고택은 사랑채와 안채가 접한 ‘ㅁ’ 자형 한옥이다,문살 하나까지 춘양목으로 지은 데서 집에 들인 공을 짐작할 수 있다,"춘양목은 뒤틀림이 적고 잘 썩지 않아 조선 시대 궁궐이나 사찰, 사대부 양반집에 ...",객실은 사랑방부터 어사방까지 총 6개,"사랑방은 두 칸 방에 사랑 대청마루를 함께 쓸 수 있고, 선비방은 아침에 가장 먼...","모든 방에 에어컨을 두었고, 방문을 열면 꽃나무가 핀 정원이 정답다",...,None,None,None,None,None,None,None,None,None,None
4197,부천문화원은 1966년에 설립되어 지금까지 40여 년 동안 부천의 전통 민속의 보존...,,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4198,강원도 홍천에서 구매한 육즙이 풍성하고 마블링이 우수한 한우 꽃등심만을 숙성시켜 참...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4199,오죽헌과 강릉시립박물관으로 구분된다,오죽헌은 조선초기의 건축물로 건축사적인 면에서 그 중요성을 인정받아 보물로 지정되었다,이곳 오죽헌 몽룡실에서 율곡 이이가 태어났다,"경내에는 오죽헌을 비롯하여 문성사 , 사랑채, 어제각 , 율곡기념관 등이 있으며,...",1962년부터 현재까지 매년 율곡을 추모하는 대현이율곡선생제곡제를 봉행하고 있다,"시립박물관은 영동지방의 민속자료와 향토 유물을 수집·보관·연구·전시하며, 전통문...",전시관은 영동지방에서 사용하던 생업도구와 의식주 관련 용품들이 전시된 향토민속관...,,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# import random
# i = 0
# while i>-1:
#   i = random.randint(0, 100)
#   j = random.randint(0, 20)
#   for k in range(j): pass
#   pass

KeyboardInterrupt: ignored

# 이제 모델 짜보자

* 참고자료 링크: https://github.com/Gyeongmin47/code_similarity/blob/master/code_submission.py

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)

In [ ]:
np.array([1, 2, 3, 4, 5]).shape

(5,)

In [ ]:
from cProfile import label


def train_model(args):
  '''
  [1. 전처리 과정에서 생성된 데이터 호출]
  [2. 텐서 생성]
  '''
  set_seed(args)

  train_data = pd.read_csv('train.csv')
  encoder = LabelEncoder()
  encoder.fit(train_data['cat3'].unique())

  N = train_data.shape[0]
  MAX_LEN = 512
  
  train_idx, valid_idx = train_test_split(np.arange(N))

  # training
  N = train_idx.shape[0]
  overview = train_data.loc[train_idx, 'overview'].values
  cat3 = encoder.transform(train_data.loc[train_idx, 'cat3'].values)

  input_ids = np.zeros((N, MAX_LEN), dtype=int)
  attention_masks = np.zeros((N, MAX_LEN), dtype=int)
  labels = np.zeros((N), dtype=int)

  tokenizer = AutoTokenizer.from_pretrained(args.checkpoint_path)

  for i in tqdm(range(N), position=0, leave=True):
    try:
      cur_ov = str(overview[i])
      encoded_input = tokenizer(cur_ov, return_tensors='pt', max_length=512,
                                padding='max_length', truncation=True)
      input_ids[i, ] = encoded_input['input_ids']
      attention_masks[i, ] = encoded_input['attention_mask']
      labels[i] = cat3[i]
    except Exception as e:
      print(e)
      pass
  
  # validating
  N = valid_idx.shape[0]

  overview = train_data.loc[valid_idx, 'overview'].values
  cat3 = encoder.transform(train_data.loc[valid_idx, 'overview'].values)

  valid_input_ids = np.zeros((N, MAX_LEN), dtype=int)
  valid_attention_masks = np.zeros((N, MAX_LEN), dtype=int)
  valid_labels = np.zeros((N), dtype=int)

  for i in tqdm(range(N), position=0, leave=True):
    try:
        cur_ov = str(overview[i])
        encoded_input = tokenizer(cur_ov, return_tensors='pt', max_length=512, padding='max_length', truncation=True)
        valid_input_ids[i, ] = encoded_input['input_ids']
        valid_attention_masks[i, ] = encoded_input['attention_mask']
        valid_labels[i] = cat3[i]
    except Exception as e:
        print(e)
        pass

    if os.path.exists(args.dir_path):
        os.makedirs(args.dir_path, exist_ok=True)
    
    print("\n\nMake tensor\n\n")
    input_ids = torch.tensor(input_ids, dtype=int)
    attention_masks = torch.tensor(attention_masks, dtype=int)
    labels= torch.tensor(labels, dtype=int)

    valid_input_ids = torch.tensor(valid_input_ids, dtype=int)
    valid_attention_masks = torch.tensor(valid_attention_masks, dtype=int)
    valid_labels = torch.tensor(valid_labels, dtype=int)


    if args.save_tensor == True:
        torch.save(input_ids, "./data/"+args.dir_path+"/train_input_ids_1012.pt")
        torch.save(attention_masks, "./data/"+args.dir_path+"train_attention_masks_1012.pt")
        torch.save(labels, "./data/"+args.dir_path+"train_labels_1012.pt")

        torch.save(valid_input_ids, "./data/"+args.dir_path+"/valid_input_ids_1012.pt")
        torch.save(attention_masks, "./data/"+args.dir_path+"/valid_attention_masks_1012.pt")
        torch.save(valid_labels, "./data/"+args.dir_path+"valid_labels_1012.pt")
    


    # Setup training
    def flat_accuracy(preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat==labels_flat)/len(labels/flat)
    
    def format_time(elapsed):
        elapsed_rounded = int(round((elapsed)))
        return str(datetime.timedelta(seconds=elapsed_rounded))
    
    train_data = TensorDataset(input_ids, attention_masks, labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.batch_size)

    validation_data = TensorDataset(valid_input_ids, valid_attention_masks, valid_labels)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=args.batch_size)

    model = AutoModelForSequenceClassification.from_pretrained(args.checkpoint_path)
    model.cuda()

    optimizer = torch.optim.AdamW(model.parameters(), lr=args.learning_rate, eps=1e-5)

    total_steps = len(train_dataloader) * args.epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    device = torch.device("cuda")
    loss_f = nn.CrossEntropyLoss()


    # Train
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []
    model.zero_grad()
    for i in range(args.epochs):
        print("")
        print('======= Epoch {:} / {:} ======'.format(i+1, args.epochs))
        print('Training...')
        t0 = time.time()
        train_loss, train_accuracy = 0, 0
        model.train()
        for step, batch in tqdm(enumerate(train_dataloader), desc="Iteration", smoothing=0.05):
            if step%10000 == 0 and not step == 0:
                elapsed = format_time(time.time() -t0)
                print('\tBatch {:>5} of {:>5}. \tElapsed: {:}.'.format(step, len(train_dataloader), elapsed))
                print('\tcurrent average loss = {}'.format(train_loss/step))

                batch = tuple(t.to(device) for t t in batch)
                b_input_ids, b_input_mask, b_labels = batch
                outputs = model(b_input_ids, attention_masks=b_input_mask, labels=b_labels)
                loss = outputs[0]
                logits = outputs[1]
                train_loss += loss.item()
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.detach().cpu().numpy()
                train_accuracy += flat_accuracy(logits, label_ids)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
        
        avg_train_loss = train_loss / len(train_dataloader)
        avg_train_accuracy = train_accuracy / len(train_dataloader)
        train_losses.append(avg_train_loss)
        train_accuracies.append(avg_train_accuracy)
        print("\tAverage training loss: {0:.8f}",format(avg_train_loss))
        print("\tAverage training accuracy: {0:.8f}".format(avg_train_accuracy))

        print("")
        print("Validating...")
        t0 = time.time()
        model.eval()
        val_loss, val_accuracy = 0, 0
        for step, batch in tqdm(enumerate(validation_dataloader), desc="Iteration", smoothing=0.05):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                outputs = model(b_input_ids, attention_masks=b_input_mask)
            
            logits = outputs[0]
            logits = logits.detach().cpu()
            labels_ids = b_labels.detach().cpu()

            logits = logits.numpy()
            label_ids = label_ids.numpy()
            val_accuracy += flat_accuracy(logits, label_ids)

        avg_val_accuracy = val_accuracy/len(validation_dataloader)
        avg_val_loss = val_loss/len(validation_dataloader)
        val_accuracies.append(avg_val_accuracy)
        val_losses.append(avg_val_loss)
        print("\tAverage validation loss: {0:.8f}".format(avg_val_loss))
        print("\tAverage validation accuracy: {0:.8f}".format(avg_val_accuracy))
        print("\tTraining epoch took: {:}".format(format_time(time.time()-t0)))

        # if np.min(val_losses) == val_losses[-1]:
        print("saving current best checkpoint")
        torch.save(model.state_dict(), "./data/{}/{}th_1012.pt".format(args.dir_path, i+1))

In [ ]:
def inference_model(args):
    test_data = pd.read_csv(PATH_MOLU)

    overview = test_data['overview'].values

    N = test_data.shape[0]
    MAX_LEN = 512

    test_input_ids = np.zeros((N, MAX_LEN), dtype=int)
    test_attention_masks = np.zeros((N, MAX_LEN), dtype=int)

    tokenizer = AutoTokenizer.form_pretrained(args.checkpoint_path)
    tokenizer.truncation_side = "right"

    for i in tqdm(range(N), position=0, leave=True):
        try:
            cur_ov = str(overview[i])
            encoded_input = tokenizer(cur_ov, return_tensors='pt', max_length=MAX_LEN, padding='max_length', truncation=True)
            test_input_ids[i, ] = encoded_input['input_ids']
            test_attention_masks[i, ] = encoded_input['attention_mask']
        
        except Exception as e:
            print(e)
            pass
    
    test_input_ids = torch.tensor(test_input_ids, dtype=int)
    test_attention_masks = torch.tensor(test_attention_masks, dtype=int)

    if args.save_tensor == True:
        torch.save(test_input_ids, "./data/{}/test_input_ids_1012.pt".format(args.dir_path))
        torch.save(test_attention_masks, "./data/{}/test_attention_masks_1012.pt".format(args.dir_path))
    
    model = AutoModelForSequenceClassification.from_pretrained(args.checkpoint_path)
    PATH = "몰루?"

    model.load_state_dict(torch.load(PATH))
    model.cuda()

    test_tensor = TensorDataset(test_input_ids, test_attention_masks)
    test_sampler = SequentialSampler(test_tensor)
    test_dataloader = DataLoader(test_tensor, sampler=test_sampler, batch_size=args.test_batch_size)

    submission = pd.read_csv("./data/sample_submission.csv")
    device = torch.device("cuda")

    preds = np.array([])
    for step